In [1]:
import pandas_datareader as pdr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

In [2]:
rawData = pdr.DataReader("SI=F", 
                       start='2000-1-1', 
                       end='2020-5-10', 
                       data_source='yahoo')

rawData.describe()

,High,Low,Open,Close,Volume,Adj Close
count,5056.000000,5056.000000,5056.000000,5056.000000,5.056000e+03,5056.000000
mean,15.134165,14.924315,15.035751,15.030289,1.669874e+05,15.030289
std,8.550389,8.352095,8.458185,8.450795,2.351958e+06,8.450795
min,4.026000,4.026000,4.026000,4.026000,0.000000e+00,4.026000
25%,7.186500,7.140000,7.169750,7.170500,3.000000e+00,7.170500
50%,15.256000,15.077500,15.182500,15.185500,3.100000e+01,15.185500
75%,18.132250,18.001000,18.087750,18.068001,1.350000e+02,18.068001
max,49.520000,47.540001,48.459999,48.584000,6.980156e+07,48.584000


In [3]:
rawData['O-C'] = rawData['Open'] - rawData['Close']

trend = []
for i in range(len(rawData)):
    if rawData['O-C'][i] >= 0:
        trend.append(1)
    else:
        trend.append(0)
rawData['Trend'] = trend

rawData['0.5(O+C)'] = ( rawData['Open'] + rawData['Close'] ) * 0.5
rawData['0.5(H+L)'] = ( rawData['High'] + rawData['Low'] ) * 0.5
rawData['AvOC-HL'] = (rawData['0.5(O+C)'] + rawData['0.5(H+L)']) * 0.5
rawData.head()

,High,Low,Open,Close,Volume,Adj Close,O-C,Trend,0.5(O+C),0.5(H+L),AvOC-HL
Date,,,,,,,,,,,
2000-02-28,5.095,5.020,5.045,5.048,14203.0,5.048,-0.003,0,5.0465,5.0575,5.05200
2000-02-29,5.090,5.035,5.065,5.048,2830.0,5.048,0.017,1,5.0565,5.0625,5.05950
2000-03-01,5.130,5.050,5.070,5.073,511.0,5.073,-0.003,0,5.0715,5.0900,5.08075
2000-03-02,5.105,5.000,5.105,5.008,645.0,5.008,0.097,1,5.0565,5.0525,5.05450
2000-03-03,5.110,5.000,5.020,5.099,307.0,5.099,-0.079,0,5.0595,5.0550,5.05725


In [4]:
av5 = []
p = 0
h = 0
for i in range(len(rawData)):
    p += 1
    for x in range(5):
        h = x+p
    av5.append(rawData[['Volume']][i:h].mean().to_numpy())

edited_av5 = av5[ 0 : len(rawData) - 6]

In [5]:
inputs = rawData[['High', 'Low', 'Open', 'Close']][ 0 : len(rawData) - 1 ].to_numpy()
inputs2 = rawData[['High', 'Low','Volume']][ 0 : len(rawData) - 1 ].to_numpy()
inputs3 = rawData['Volume'][ 0 : len(rawData) - 1 ].to_numpy()
inputs4  = edited_av5

targets = rawData[['Trend']][ 6 : len(rawData) ].to_numpy()

scaler = StandardScaler()
# scaler.fit(inputs)
# scaler.fit(inputs2)
# scaler.fit(inputs3.reshape(-1,1))
scaler.fit(inputs4)

# scaled_inputs = scaler.transform(inputs)
# scaled_inputs2 = scaler.transform(inputs2)
# scaled_inputs3 = scaler.transform(inputs3.reshape(-1,1))
scaled_inputs4 = scaler.transform(inputs4)


scaled_inputs4

array([[-0.08728493],
       [-0.08891077],
       [-0.0892072 ],
       ...,
       [ 1.60323467],
       [ 3.98858298],
       [ 6.23862759]])

In [6]:
sum_uptrend = int(np.sum(targets))

sum_downtrend = 0

unnecessary_indices = []

for i in range(len(targets)):
    if targets[i] == 0:
        sum_downtrend += 1
        if sum_downtrend > sum_uptrend:
            unnecessary_indices.append(i)

extracted_inputs = np.delete( scaled_inputs4, unnecessary_indices, axis = 0 )
extracted_targets = np.delete( targets, unnecessary_indices, axis = 0 )
print(len(extracted_inputs), len(extracted_targets), unnecessary_indices)

5050 5050 []


In [7]:
shuffle_value = np.arange(len(extracted_inputs))
np.random.shuffle(shuffle_value)

shuffled_inputs = extracted_inputs[shuffle_value]
shuffled_targets = extracted_targets[shuffle_value]
shuffled_inputs

array([[-0.08937807],
       [-0.08939669],
       [-0.08941461],
       ...,
       [-0.08617506],
       [-0.08930847],
       [11.81152312]])

In [8]:
total_samples = len(shuffled_inputs)

train_samples = int( 0.8 * total_samples )
validation_samples = int( 0.1 * total_samples )
test_samples = total_samples - ( train_samples + validation_samples )

train_inputs = shuffled_inputs[ :  train_samples]
train_targets = shuffled_targets[ : train_samples]

validation_inputs = shuffled_inputs[ train_samples : train_samples + validation_samples ]
validation_targets = shuffled_targets[ train_samples : train_samples + validation_samples ]

test_inputs = shuffled_inputs[ train_samples + validation_samples : ]
test_targets = shuffled_targets[ train_samples + validation_samples : ]

print( np.sum(train_targets), train_samples, np.sum(train_targets) /  train_samples )
print( np.sum(validation_targets), validation_samples, np.sum(validation_targets) /  validation_samples )
print( np.sum(test_targets), test_samples, np.sum(test_targets) /  test_samples )

2705 4040 0.6695544554455446
346 505 0.6851485148514852
339 505 0.6712871287128713


In [9]:
np.savez('silver_train', inputs=train_inputs, targets=train_targets)
np.savez('silver_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('silver_test', inputs=test_inputs, targets=test_targets)

In [10]:
npz_trn = np.load('silver_train.npz')
trn_inputs, trn_targets = npz_trn['inputs'].astype(np.float), npz_trn['targets'].astype(np.int)

npz_val = np.load('silver_validation.npz')
val_inputs, val_targets = npz_val['inputs'].astype(np.float), npz_val['targets'].astype(np.int)

npz_tst = np.load('silver_test.npz')
tst_inputs, tst_targets = npz_tst['inputs'].astype(np.float), npz_tst['targets'].astype(np.int)

In [11]:
train_inputs

array([[-0.08937807],
       [-0.08939669],
       [-0.08941461],
       ...,
       [-0.08942154],
       [-0.08929136],
       [-0.0894175 ]])

In [12]:
input_size = 1
output_size = 2

hidden_layer_size = 100

model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), 
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), 
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), 
    tf.keras.layers.Dense(output_size, activation='softmax') 
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

batch_size = 100

epochs = 50

model.fit(trn_inputs,
          trn_targets,
          batch_size=batch_size, 
          epochs=epochs,
          validation_data=(val_inputs, val_targets),
          verbose = 2 
          )  

Train on 4040 samples, validate on 505 samples
Epoch 1/50
4040/4040 - 1s - loss: 0.6501 - accuracy: 0.6696 - val_loss: 0.6258 - val_accuracy: 0.6851
Epoch 2/50
4040/4040 - 0s - loss: 0.6350 - accuracy: 0.6696 - val_loss: 0.6243 - val_accuracy: 0.6851
Epoch 3/50
4040/4040 - 0s - loss: 0.6347 - accuracy: 0.6696 - val_loss: 0.6227 - val_accuracy: 0.6851
Epoch 4/50
4040/4040 - 0s - loss: 0.6348 - accuracy: 0.6696 - val_loss: 0.6288 - val_accuracy: 0.6851
Epoch 5/50
4040/4040 - 0s - loss: 0.6354 - accuracy: 0.6696 - val_loss: 0.6228 - val_accuracy: 0.6851
Epoch 6/50
4040/4040 - 0s - loss: 0.6346 - accuracy: 0.6696 - val_loss: 0.6232 - val_accuracy: 0.6851
Epoch 7/50
4040/4040 - 0s - loss: 0.6356 - accuracy: 0.6696 - val_loss: 0.6240 - val_accuracy: 0.6851
Epoch 8/50
4040/4040 - 0s - loss: 0.6346 - accuracy: 0.6696 - val_loss: 0.6229 - val_accuracy: 0.6851
Epoch 9/50
4040/4040 - 0s - loss: 0.6356 - accuracy: 0.6696 - val_loss: 0.6278 - val_accuracy: 0.6851
Epoch 10/50
4040/4040 - 0s - loss: 

In [13]:
test_loss, test_accuracy = model.evaluate(tst_inputs, tst_targets)

505/505 [==============================] - 0s 75us/sample - loss: 0.6335 - accuracy: 0.6713
